In [35]:
import os
import rioxarray as rxr
import xarray as xr
from dataclasses import asdict
from dscreator.cfarray.attributes import DatasetAttrsGrid, VariableAttrs
from datetime import datetime
from dscreator import utils
from dscreator.storage import BaseHandler, get_storage_handler
from dscreator.sources.ferrybox.extractor import TrajectoryExtractor
from sqlalchemy import create_engine
from dscreator.config import SETTINGS
from dscreator.datasets.trajectories.ferrybox import FerryboxTrajBuilder

In [36]:
boat = "FA"
measurement_parameters = ["Temperature", "Salinity", "Oxygen"]
uuid="29b7de62-e1fa-4dce-90e4-7ff8a0931397",
datasetname="FA_ferrybox_2017_2022",
stationname="Color Fantasy",
projectname="NorSoop",

In [37]:
engine = create_engine(SETTINGS.database_url)
te = TrajectoryExtractor(engine, boat, measurement_parameters)
ts = te.fetch_slice(start_time=datetime(2021, 12, 12, 16, 0, 0), end_time=datetime(2021, 12, 13, 16, 0, 0))
print(f"Fetched data ts")

Fetched data ts


In [38]:
print(f"Manual qc is not yet implemented")
# print(f"Data after manual qc {ts}")

Manual qc is not yet implemented
Data after manual qc NamedTrajectory(array_list=[NamedArray(variable_name='Temperature', values=[6.468, 6.456, 6.459, 6.467, 6.451, 6.444, 6.46, 6.441, 6.44, 6.372, 6.329, 6.274, 6.264, 6.316, 6.306, 6.313, 6.335, 6.367, 6.392, 6.44, 6.439, 6.438, 6.361, 6.359, 6.376, 6.408, 6.463, 6.48, 6.488, 6.515, 6.535, 6.553, 6.578, 6.585, 6.572, 6.592, 6.581, 6.574, 6.577, 6.572, 6.595, 6.584, 6.544, 6.51, 6.462, 6.47, 6.436, 6.42, 6.431, 6.434, 6.426, 6.37, 6.38, 6.361, 6.367, 6.354, 6.366, 6.323, 6.289, 6.313, 6.325, 6.367, 6.38, 6.329, 6.162, 6.163, 6.199, 6.257, 6.341, 6.34, 6.346, 6.35, 6.326, 6.073, 6.038, 6.007, 5.996, 5.995, 5.94, 5.94, 5.946, 5.951, 5.92, 5.874, 5.813, 5.778, 5.762, 5.799, 5.829, 5.861, 6.084, 6.198, 6.218, 5.939, 5.793, 5.754, 5.753, 5.797, 5.831, 5.942, 5.958, 5.957, 5.991, 5.978, 6.005, 5.975, 5.962, 6.033, 6.055, 6.069, 6.095, 6.093, 6.081, 6.201, 6.259, 6.281, 6.33, 6.379, 6.411, 6.43, 6.399, 6.307, 6.164, 6.161, 6.242, 6.159, 6.181

In [39]:
tb = FerryboxTrajBuilder(
    uuid=uuid,
    dataset_name=datasetname,
    station_name=stationname,
    project_name=projectname,
    is_acdd=True,
)
ds = tb.create(ts)
print(f"ds {ds}")

ds <xarray.Dataset>
Dimensions:                (time: 965)
Coordinates:
  * time                   (time) datetime64[ns] 2021-12-12T16:00:12 ... 2021...
    longitude              (time) float64 10.92 10.93 10.93 ... 10.64 10.64
    latitude               (time) float64 54.95 54.96 54.96 ... 59.12 59.11
Data variables:
    sea_water_temperature  (time) float64 6.468 6.456 6.459 ... 5.403 5.245 5.22
    salinity               (time) float64 9.928 9.906 9.857 ... 5.671 5.687 5.69
    oxygen                 (time) float64 93.88 95.03 93.84 ... 94.12 94.74
    trajectory_name        object ('Color Fantasy',)
Attributes: (12/36)
    featureType:             trajectory
    id:                      ('29b7de62-e1fa-4dce-90e4-7ff8a0931397',)
    title:                   Ferrybox on MS Color Fantasy
    summary:                 Ferry sailing from Oslo to Kiel. For more inform...
    date_created:            2023-04-12 14:47:21.969864
    keywords:                
    ...                      ...

In [44]:
if ds.dims["time"] > 0:
    print(f"Saving dataset slice {ds.time[0].values} --> {ds.time[-1].values}")

    sh= get_storage_handler(
        project_name=str(datasetname),
        dataset_name=str(stationname),
        unlimited_dims=["time"],
        filename_prefix= "FA"
    )
    print(f"sh {sh}")
    print(ds)
    sh.save_dataset(ds)
else:
    print("Found no data for interval")





Saving dataset slice 2021-12-12T16:00:12.000000000 --> 2021-12-13T15:59:47.000000000
sh LocalStorageHandler(project_name="('FA_ferrybox_2017_2022',)", dataset_name="('Color Fantasy',)", filename_prefix='FA', workdir="datasets/('fa_ferrybox_2017_2022',)/('Color Fantasy',)", restart_filepath="datasets/('fa_ferrybox_2017_2022',)/('Color Fantasy',)/.('Color Fantasy',).restart.json", unlimited_dims=['time'])
<xarray.Dataset>
Dimensions:                (time: 965)
Coordinates:
  * time                   (time) datetime64[ns] 2021-12-12T16:00:12 ... 2021...
    longitude              (time) float64 10.92 10.93 10.93 ... 10.64 10.64
    latitude               (time) float64 54.95 54.96 54.96 ... 59.12 59.11
Data variables:
    sea_water_temperature  (time) float64 6.468 6.456 6.459 ... 5.403 5.245 5.22
    salinity               (time) float64 9.928 9.906 9.857 ... 5.671 5.687 5.69
    oxygen                 (time) float64 93.88 95.03 93.84 ... 94.12 94.74
    trajectory_name        object ('C

ValueError: setting an array element with a sequence